# StepFunctions Data Science SDK for BYO Container

Here we demonstrate how to use the StepFunction Data Science SDK to do an end-to-end data science workflow where you bring your own code and deploy a model on SageMaker.

The Steps are as follows:

1. Create a Lambda function which launches a CodeBuild job that launches the creation of your Docker container. (Steps for this are included separately)
2. Launch the Lambda function as a Step Functions workflow. 
3. Once the Docker container is built, launch a SageMaker training job using SF DS SDK.
4. Use the DS SDK to deploy the trained model.

## Create an execution role for Step Functions

You need an execution role so that you can create and execute workflows in Step Functions.

1. Go to the IAM console
2. Select Roles and then Create role.
3. Under Choose the service that will use this role select Step Functions
4. Choose Next until you can enter a Role name
5. Enter a name such as StepFunctionsWorkflowExecutionRole and then select Create role

Attach a policy to the role you created. The following steps attach a policy that provides full access to Step Functions, however as a good practice you should only provide access to the resources you need.

1. Under the Permissions tab, click Add inline policy
2. Enter the following in the JSON tab

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker:CreateTransformJob",
                "sagemaker:DescribeTransformJob",
                "sagemaker:StopTransformJob",
                "sagemaker:CreateTrainingJob",
                "sagemaker:DescribeTrainingJob",
                "sagemaker:StopTrainingJob",
                "sagemaker:CreateHyperParameterTuningJob",
                "sagemaker:DescribeHyperParameterTuningJob",
                "sagemaker:StopHyperParameterTuningJob",
                "sagemaker:CreateModel",
                "sagemaker:CreateEndpointConfig",
                "sagemaker:CreateEndpoint",
                "sagemaker:DeleteEndpointConfig",
                "sagemaker:DeleteEndpoint",
                "sagemaker:UpdateEndpoint",
                "sagemaker:ListTags",
                "lambda:InvokeFunction",
                "sqs:SendMessage",
                "sns:Publish",
                "ecs:RunTask",
                "ecs:StopTask",
                "ecs:DescribeTasks",
                "dynamodb:GetItem",
                "dynamodb:PutItem",
                "dynamodb:UpdateItem",
                "dynamodb:DeleteItem",
                "batch:SubmitJob",
                "batch:DescribeJobs",
                "batch:TerminateJob",
                "glue:StartJobRun",
                "glue:GetJobRun",
                "glue:GetJobRuns",
                "glue:BatchStopJobRun"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "iam:PassRole"
            ],
            "Resource": "*",
            "Condition": {
                "StringEquals": {
                    "iam:PassedToService": "sagemaker.amazonaws.com"
                }
            }
        },
        {
            "Effect": "Allow",
            "Action": [
                "events:PutTargets",
                "events:PutRule",
                "events:DescribeRule"
            ],
            "Resource": [
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTrainingJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTransformJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTuningJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForECSTaskRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForBatchJobsRule"
            ]
        }
    ]
}
```

1. Choose Review policy and give the policy a name such as StepFunctionsWorkflowExecutionPolicy
2. Choose Create policy. You will be redirected to the details page for the role.
2. Note the Role ARN at the top of the Summary, as you'll need it later.


## Import necessary libraries

In [163]:
# Import required libraries and upload the training data to S3
import boto3
import os
import numpy as np
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
import time
import uuid



role = get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket() # or feel free to replace with a bucket of your choosing
WORK_DIRECTORY = 'PennFudanPed'
key = 'BYO-Mask-RCNN'
prefix = 'key/{}'.format(key, WORK_DIRECTORY)

In [164]:
print(bucket)
print(WORK_DIRECTORY)
print(key)
print(prefix)

sagemaker-us-east-1-243645717177
PennFudanPed
BYO-Mask-RCNN
key/BYO-Mask-RCNN


In [165]:
# install StepFunctions SDK
import sys
!{sys.executable} -m pip install --upgrade stepfunctions

Requirement already up-to-date: stepfunctions in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (1.0.0.6)
Requirement not upgraded as not directly required: sagemaker>=1.42.8 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from stepfunctions) (1.50.9.post0)
Requirement not upgraded as not directly required: pyyaml in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from stepfunctions) (5.2)
Requirement not upgraded as not directly required: boto3>=1.9.213 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from stepfunctions) (1.11.14)
Requirement not upgraded as not directly required: importlib-metadata>=1.4.0 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from sagemaker>=1.42.8->stepfunctions) (1.5.0)
Requirement not upgraded as not directly required: smdebug-rulesconfig==0.1.2 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from sagemaker>=1.42.8->stepfunctions) 

## Find your StepFunctionsWorkflowExecutionRole ARN

Go back to IAM and find the ARN of the role where you created the inline policy during the setup of the notebook instance. 

You can find the full ARN in the top of that IAM page.

In [166]:
import stepfunctions
import logging
from stepfunctions.steps import (LambdaStep, Retry, Catch, Fail, Chain, TrainingStep, ModelStep, EndpointConfigStep, EndpointStep)
from stepfunctions.workflow import Workflow
from stepfunctions.template.pipeline import TrainingPipeline
from stepfunctions.inputs import ExecutionInput

stepfunctions.set_stream_logger(level=logging.INFO)

# paste the StepFunctionsWorkflowExecutionRole ARN from above in "<>"
workflow_execution_role = "arn:aws:iam::243645717177:role/StepFunctionsWorkflowExecutionRole" 

## Download the training data

In [167]:
!rm -rf PennFudanPed*
!wget "https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip"
!unzip PennFudanPed.zip

--2020-02-24 14:18:48--  https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip
Resolving www.cis.upenn.edu (www.cis.upenn.edu)... 158.130.69.163, 2607:f470:8:64:5ea5::d
Connecting to www.cis.upenn.edu (www.cis.upenn.edu)|158.130.69.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53723336 (51M) [application/zip]
Saving to: ‘PennFudanPed.zip’

PennFudanPed.zip    100%[===================>]  51.23M  4.73MB/s    in 9.0s    

2020-02-24 14:18:57 (5.67 MB/s) - ‘PennFudanPed.zip’ saved [53723336/53723336]

Archive:  PennFudanPed.zip
   creating: PennFudanPed/
  inflating: PennFudanPed/added-object-list.txt  
   creating: PennFudanPed/Annotation/
  inflating: PennFudanPed/Annotation/FudanPed00001.txt  
  inflating: PennFudanPed/Annotation/FudanPed00002.txt  
  inflating: PennFudanPed/Annotation/FudanPed00003.txt  
  inflating: PennFudanPed/Annotation/FudanPed00004.txt  
  inflating: PennFudanPed/Annotation/FudanPed00005.txt  
  inflating: PennFudanPed/Annotati

  inflating: PennFudanPed/PNGImages/FudanPed00009.png  
  inflating: PennFudanPed/PNGImages/FudanPed00010.png  
  inflating: PennFudanPed/PNGImages/FudanPed00011.png  
  inflating: PennFudanPed/PNGImages/FudanPed00012.png  
  inflating: PennFudanPed/PNGImages/FudanPed00013.png  
  inflating: PennFudanPed/PNGImages/FudanPed00014.png  
  inflating: PennFudanPed/PNGImages/FudanPed00015.png  
  inflating: PennFudanPed/PNGImages/FudanPed00016.png  
  inflating: PennFudanPed/PNGImages/FudanPed00017.png  
  inflating: PennFudanPed/PNGImages/FudanPed00018.png  
  inflating: PennFudanPed/PNGImages/FudanPed00019.png  
  inflating: PennFudanPed/PNGImages/FudanPed00020.png  
  inflating: PennFudanPed/PNGImages/FudanPed00021.png  
  inflating: PennFudanPed/PNGImages/FudanPed00022.png  
  inflating: PennFudanPed/PNGImages/FudanPed00023.png  
  inflating: PennFudanPed/PNGImages/FudanPed00024.png  
  inflating: PennFudanPed/PNGImages/FudanPed00025.png  
  inflating: PennFudanPed/PNGImages/FudanPed0002

  inflating: PennFudanPed/PNGImages/PennPed00085.png  
  inflating: PennFudanPed/PNGImages/PennPed00086.png  
  inflating: PennFudanPed/PNGImages/PennPed00087.png  
  inflating: PennFudanPed/PNGImages/PennPed00088.png  
  inflating: PennFudanPed/PNGImages/PennPed00089.png  
  inflating: PennFudanPed/PNGImages/PennPed00090.png  
  inflating: PennFudanPed/PNGImages/PennPed00091.png  
  inflating: PennFudanPed/PNGImages/PennPed00092.png  
  inflating: PennFudanPed/PNGImages/PennPed00093.png  
  inflating: PennFudanPed/PNGImages/PennPed00094.png  
  inflating: PennFudanPed/PNGImages/PennPed00095.png  
  inflating: PennFudanPed/PNGImages/PennPed00096.png  
  inflating: PennFudanPed/readme.txt  


## Create an S3 bucket for the training dataset

- Go to S3 service
- Create a new bucket in the us-east-1 region, note the name to use it below

## Upload the training dataset to Amazon S3

- Update the `bucket` variable below with the bucket name you created above;
- The first variable is created after the `unzip` command 2 cells above;
- the `key_prefix` variable can remain unchanged.

After executing the cell below, wait until the `data_location` variable is printed, or until the S3 bucket is populated. You can check this by opening S3 in another browser tab.

In [169]:
data_location = sess.upload_data(WORK_DIRECTORY, bucket=bucket, key_prefix=prefix)
print(data_location)

s3://sagemaker-us-east-1-243645717177/key/BYO-Mask-RCNN


# Define Estimator and StepFunctions Workflow

In [238]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

# Note that this image name will work below provided you have made the changes to the Environment variables in the Lambda
# defintion as suggested in the workshop Readme. If not please make those first.

image = '{}.dkr.ecr.{}.amazonaws.com/sm-container-maskrcnn:torch'.format(account, region) 

maskrcnn = sagemaker.estimator.Estimator(image,
                       role, 1, 'ml.p3.16xlarge', # --- ml.p3.2xlarge --- feel free to modify with your own. A cost estimate is provided in Readme.
                       output_path="s3://{}/{}/output".format(sess.default_bucket(), key),
                       sagemaker_session=sess)

maskrcnn.set_hyperparameters(num_epochs = 1,
                              num_classes = 2)

#maskrcnn.fit(os.path.dirname(data_location))

### Create StepFunction Pipeline

Remember to replace the `FunctionName` variable below to the Lambda function you created through Cloudformation.

In [239]:
lambda_state = LambdaStep(
    state_id="Calls CodeBuild to Build Container",
    parameters={  
        "FunctionName": "lambda-build-docker", #replace with the name of the Lambda function you created
        "Payload": {  
           "input": "HelloWorld"
        }
    }
)

lambda_state.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=15,
    max_attempts=2,
    backoff_rate=4.0
))

lambda_state.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("LambdaTaskFailed")
))

In [240]:
execution_input = ExecutionInput(schema={
    'JobName': str, 
    'ModelName': str,
    'EndpointName': str
})

In [241]:
train_step = TrainingStep(
    'Train Step', 
    estimator=maskrcnn,
#    role=workflow_execution_role,
    data=os.path.dirname(data_location),
    job_name=execution_input['JobName']
)

In [242]:
model_step = ModelStep(
    'Save model',
    model=train_step.get_expected_model(),
    model_name=execution_input['ModelName'] 
)

In [243]:
endpoint_config_step = EndpointConfigStep(
    "Create Endpoint Config",
    endpoint_config_name=execution_input['ModelName'],
    model_name=execution_input['ModelName'],
    initial_instance_count=1,
    instance_type='ml.m5.large'
)

In [244]:
endpoint_step = EndpointStep(
    "Create Endpoint",
    endpoint_name=execution_input['EndpointName'],
    endpoint_config_name=execution_input['ModelName']
)

In [245]:
workflow_definition = Chain([
    lambda_state,
    train_step,
    model_step,
    endpoint_config_step,
    endpoint_step
])

# Next, we define the workflow
workflow = Workflow(
    name="MyWorkflow-BYOC-MaskRCNN-{}".format(uuid.uuid1().hex),
    definition=workflow_definition,
    role=workflow_execution_role
)

In [246]:
print(workflow.definition.to_json(pretty=True))

{
    "StartAt": "Calls CodeBuild to Build Container",
    "States": {
        "Calls CodeBuild to Build Container": {
            "Parameters": {
                "FunctionName": "lambda-build-docker",
                "Payload": {
                    "input": "HelloWorld"
                }
            },
            "Resource": "arn:aws:states:::lambda:invoke",
            "Type": "Task",
            "Next": "Train Step",
            "Retry": [
                {
                    "ErrorEquals": [
                        "States.TaskFailed"
                    ],
                    "IntervalSeconds": 15,
                    "MaxAttempts": 2,
                    "BackoffRate": 4.0
                }
            ],
            "Catch": [
                {
                    "ErrorEquals": [
                        "States.TaskFailed"
                    ],
                    "Next": "LambdaTaskFailed"
                }
            ]
        },
        "Train Step": {
            "Reso

In [247]:
workflow.render_graph()

In [248]:
workflow.create()

[INFO] Workflow created successfully on AWS Step Functions.


'arn:aws:states:us-east-1:243645717177:stateMachine:MyWorkflow-BYOC-MaskRCNN-b9db269e571c11eaba0315ecb0df2130'

In [249]:
execution = workflow.execute(
    inputs={
        'JobName': 'BYOC-Mask-RCNN-{}'.format(uuid.uuid1().hex), # Each Sagemaker Job requires a unique name
        'ModelName': 'BYOC-Mask-RCNN-{}'.format(uuid.uuid1().hex), # Each Model requires a unique name,
        'EndpointName': 'BYOC-Mask-RCNN-{}'.format(uuid.uuid1().hex) # Each Endpoint requires a unique name,
    }
)

[INFO] Workflow execution started successfully on AWS Step Functions.


## Watch the progress of your workflow here

In [253]:
from IPython.display import display, display_html
from stepfunctions.workflow import ExecutionStatus

while execution.status == ExecutionStatus.Running:
    display_html(execution.render_progress())
    time.sleep(30)

True


## Inferences

Once your model is deployed, you can run inferences using this endpoint by using the SageMaker RealTimePredictor API. Please refer to the existing SageMaker documentation for how to do this.

Also to ensure you don't rack up costs, make sure you delete the endpoint once you are done. 

## End